In [1]:
#!pip install --upgrade xlrd

In [2]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from tqdm import tqdm
import seaborn as sns


In [3]:
df_train = pd.read_excel('/content/Lab_Data.xls', sheet_name='2004--2005 Data')
df_train= np.array(df_train, np.float64)
X_train, y_train = df_train[:,1:], df_train[:,0]

FileNotFoundError: [Errno 2] No such file or directory: '/content/Lab_Data.xls'

In [ ]:
class LogisticRegression:
    def __init__(self, learningRate, tolerance, maxIteration=5000, index=[], yes_remove_index=False):
        self.learningRate=learningRate
        self.tolerance=tolerance
        self.maxIteration=maxIteration
        self.index=index
        self.yes_remove_index=yes_remove_index
        
    def Read_values(self):
        df_train = pd.read_excel('Lab_Data.xls', sheet_name='2004--2005 Data')
        df_test = pd.read_excel('Lab_Data.xls', sheet_name='2004--2007 Data')
        df_train,df_test  = np.array(df_train, dtype=np.float64), np.array(df_test, dtype=np.float64)
        
        X_train, y_train = df_train[:,1:], df_train[:,0]
        X_test, y_test = df_test[:,1:], df_test[:,0]
        return X_train, y_train, X_test, y_test
    
    def remove_index(self, X_train, y_train, index):
        print(index)
        X_train=np.delete(X_train, index, 0)
        y_train=np.delete(y_train, index, 0)
        return X_train, y_train
    
    def addX0(self,X):
        return np.column_stack([np.ones([X.shape[0],1]),X])
    
    def sigmoid(self,z):
        sig=1/(1+np.exp(-z))
        return sig
    
    def costFunction(self,X,y):
        sig=self.sigmoid(X.dot(self.w))
        pred=y*np.log(sig)+(1-y)*np.log(1-sig)
        cost=pred.sum()
        return cost
    
    def gradient(self, X,y):
        sig=self.sigmoid(X.dot(self.w))
        grad=(sig-y).dot(X)
        return grad
    
    def gradientDescent(self,X,y):
        cost=[]
        last=float('inf')
        for i in tqdm(range(self.maxIteration)):
            self.w = self.w - self.learningRate*(self.gradient(X,y))
            current_cost= self.costFunction(X,y)
            diff=last-current_cost
            #last=cur_cost
            cost.append(abs(current_cost))
            
            if diff<self.tolerance:
                print("the model stopped. Converged at {}".format(i))
                break
        self.plotCost(cost)
        return
    
    def plotCost(self, costSequence):
        axis=list(range(0, len(costSequence),1))
        fig, ax = plt.subplots()
        ax.plot(axis, costSequence)
        ax.set_xlabel('Iterations')
        ax.set_ylabel('Cost Function')
        ax.set_title('Cost Function')
        ax.grid()
        plt.show()
        
    def plot(self):
        plt.figure(figsize=(12, 8))
        ax = plt.axes(projection='3d')

        # Data for three-dimensional scattered points
        ax.scatter3D(self.X_rtrain[:, 0], self.X_rtrain[:, 1], 
                     self.sigmoid(self.X_rtrain.dot(self.w)), 
                     c = self.y_rtrain[:], cmap='viridis', s=100);

        ax.set_xlim3d(55, 80)
        ax.set_ylim3d(80, 240)
        plt.xlabel('$x_1$ feature', fontsize=15)
        plt.ylabel('$x_2$ feature', fontsize=15, )
        ax.set_zlabel('$P(Y = 1|x_1, x_2)$', fontsize=15, rotation = 0)

    def scatterPlt(self):
            # evenly sampled points
            x_min, x_max = 55, 80
            y_min, y_max = 80, 240

            xx, yy = np.meshgrid(np.linspace(x_min, x_max, 250),
                                 np.linspace(y_min, y_max, 250))
            grid = np.c_[xx.ravel(), yy.ravel()]
            probs = grid.dot(self.w).reshape(xx.shape)

            f, ax = plt.subplots(figsize=(14,12))


            ax.contour(xx, yy, probs, levels=[0.5], cmap="Greys", vmin=0, vmax=.6)


            ax.scatter(self.X_rtrain[:, 0], self.X_rtrain[:, 1], 
                       c=self.y_rtrain[:], s=50,
                       cmap="RdBu", vmin=-.2, vmax=1.2,
                       edgecolor="white", linewidth=1)

            plt.xlabel('x1 feature')
            plt.ylabel('x2 feature')


    def plot3D(self):
            # evenly sampled points
            x_min, x_max = 55, 80
            y_min, y_max = 80, 240

            xx, yy = np.meshgrid(np.linspace(x_min, x_max, 250),
                                 np.linspace(y_min, y_max, 250))

            grid = np.c_[xx.ravel(), yy.ravel()]
            probs = grid.dot(self.w).reshape(xx.shape)
            fig = plt.figure(figsize=(14,12))
            ax = plt.axes(projection='3d')
            ax.contour3D(xx, yy, probs, 50, cmap='binary')

            ax.scatter3D(self.X_rtrain[:, 0], self.X_rtrain[:, 1], 
                       c=self.y_rtrain[:], s=50,
                       cmap="RdBu", vmin=-.2, vmax=1.2,
                       edgecolor="white", linewidth=1)

            ax.set_xlabel('x1')
            ax.set_ylabel('x2')
            ax.set_zlabel('probs')
            ax.set_title('3D contour')
            plt.show()
    
    def predict(self, X):
        sig = self.sigmoid(X.dot(self.w))
        return np.around(sig)
    
    def evaluate(self, y, y_hat):
        y = (y==1)
        y_hat = (y_hat==1)
        accuracy= (y==y_hat).sum()/y.size
        precision = (y&y_hat).sum()/y_hat.sum()
        recall=(y&y_hat).sum()/y.sum()
        return accuracy, precision, recall
    
    def run_model(self):
        self.X_train, self.y_train, self.X_test, self.y_test = self.Read_values()
        
        if self.yes_remove_index== True:
            self.X_rtrain, self.y_rtrainr= self.remove_index(self.X_train, self.y_train, self.index)
        elif self.yes_remove_index==False:
            self.X_rtrain, self.y_rtrain = self.X_train, self.y_train
        
        self.w=np.ones(self.X_rtrain.shape[1], dtype=np.float64)*0
        self.gradientDescent(self.X_rtrain, self.y_rtrain)
        print('The parameters are {}'.format(self.w))
        
        y_hat_train=self.predict(self.X_rtrain)
        accuracy, precision, recall = self.evaluate(self.y_rtrain, y_hat_train)
       
        print("Accuracy", accuracy)
        print('Precision', precision)
        print("Recall", recall)
        
       # y_hat_test=self.predict(self.X_test)
       # accuracy_test, precision_test, recall_test = self.evaluate(self.y_test, y_hat_test)
        
        #print("\nEvaluation for test data: \n")
        #print("Accuracy", accuracy_test)
        #print('Precision', precision_test)
        #print("Recall", recall_test)
       
        self.scatterPlt()
        self.plot()
        self.plot3D()
        
        return
        
        
    

In [ ]:
lr=LogisticRegression(learningRate=0.000001,tolerance=0.0000005)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
lr.run_model()

In [4]:
lr2=LogisticRegression(learningRate=0.000001,tolerance=0.0000005,index=[0,38],yes_remove_index=True)
lr2.run_model()

NameError: name 'LogisticRegression' is not defined

Observations:
We can see that accuracy, precision, recall all are increased by removing 2 datapoints that means the degree of misclassification is decreased. 
As Recall is 1, we can say that number of False Negatives are zero after removing 2 data points(Recall = TP/TP+FN)
Also, our model classified all of the data points correctly after removing 2 data points. 